# **Obtención y preparación de datos**

# OD15. Reindexación de Estructuras en Pandas

Creación de una copia de una estructura pandas -una serie o un dataframe- en base a un nuevo índice.

In [ ]:
import numpy as np
import pandas as pd

## <font color='blue'>**Reindexación de series**</font>

El método básico para la reindexación de series es `pandas.Series.reindex`. Este método devuelve una copia de una serie basándose en el índice modificado de la serie original.

In [ ]:
s = pd.Series([1, 2, 3, 4, 5], index = ["a", "c", "f", "g", "j"])
s

,0
a,1
c,2
f,3
g,4
j,5


Se trata de una serie cuyas etiquetas son letras no consecutivas. Es posible generar una copia reindexada de esta serie de la siguiente forma:



In [ ]:
r = s.reindex(["g", "c", "a", "j", "f"])
r

,0
g,4
c,2
a,1
j,5
f,3


El primer argumento siempre es el nuevo índice. En el caso anterior se trata de una versión desordenada del índice original, por lo que la serie generada es también una versión desordenada de la original.

Si el nuevo índice es un subconjunto del original, la serie generada no contendrá todos los valores de la serie de la que se patió, tan solo los incluidos en el nuevo índice. En el siguiente ejemplo, el nuevo índice no incluye la etiqueta "a" por lo que la serie generada no incluye el valor correspondiente (1):

In [ ]:
r = s.reindex(["g", "c", "j", "f"])
r

,0
g,4
c,2
j,5
f,3


Por el contrario, si en el nuevo índice se incluyen etiquetas no incluidas en el índice original, la nueva serie incluirá dicha etiqueta pero el valor asignado a ella recibe el valor por defecto `NaN`. En este próximo ejemplo incluimos la etiqueta $e$ (no presente en el índice original) en el nuevo índice:

In [ ]:
r = s.reindex(["g", "c", "e", "a", "j", "f"])
r

,0
g,4.0
c,2.0
e,NaN
a,1.0
j,5.0
f,3.0


Este valor de relleno (`NaN`) es personalizable usando el parámetro `fill_value`. Si repetimos las instrucciones anteriores especificando que el valor de relleno sea, por ejemplo, 0:

In [ ]:
r = s.reindex(["g", "c", "e", "a", "j", "f"], fill_value = 0)
r

,0
g,4
c,2
e,0
a,1
j,5
f,3


Una alternativa a fijar el valor de relleno por defecto es aplicar "lógica de relleno", rellenando los valores inexistentes con otro valor que sí exista. Tenemos tres opciones:

## <font color='blue'>**Forward fill**</font>

La primera opción consiste en rellenar los valores inexistentes "hacia adelante", haciendo que los valores existentes rellenen los valores inexistentes que los sigan. O, en otras palabras, rellenar los valores inexistentes con el primer valor existente que los precedan:

In [ ]:
s = pd.Series([1, 2, 3, 4, 5], index = ["a", "c", "f", "g", "j"])
s

,0
a,1
c,2
f,3
g,4
j,5


In [ ]:
r = s.reindex(["g", "c", "e", "a", "j", "f"], method = "ffill")
r

,0
g,4
c,2
e,2
a,1
j,5
f,3


En este caso, el valor correspondiente a la etiqueta $e$ se rellena con el valor de la etiqueta anterior $c$. Pero no la anterior en el nuevo índice, sino la anterior en el índice original. Veámoslos con otro ejemplo:

In [ ]:
r = s.reindex(["g", "c", "m", "a", "j", "f"], method = "ffill")
r

,0
g,4
c,2
m,5
a,1
j,5
f,3


Ahora, la etiqueta nueva es $m$, siendo precedida en el índice original por la $j$ (si se ordenan alfabéticamente), por lo que el valor que recibe `r["m"]` es el que tenía `r["j"]: 5`.

## <font color='blue'>**Backward fill**</font>

En este otro caso, los valores inexistentes se rellenan "hacia atrás", con el primer valor existente que los siga.

In [ ]:
s = pd.Series([1, 2, 3, 4, 5], index = ["a", "c", "f", "g", "j"])
s

,0
a,1
c,2
f,3
g,4
j,5


In [ ]:
r = s.reindex(["g", "c", "e", "a", "j", "f"], method = "bfill")
r

,0
g,4
c,2
e,3
a,1
j,5
f,3


Nuevamente, la etiqueta no existente en el índice original es $e$, y el valor que se le asigna es el correspondiente a la etiqueta que seguía a $e$ en dicho índice (si se ordenan alfabéticamente): $f$. Por lo tanto, se asigna a `r["e"]` el valor de `r["f"]: 3`.

## <font color='blue'>**Nearest (el más cercano)**</font>

La tercera opción asigna a cada valor desconocido el valor más próximo en la serie original. Para ver esta opción en funcionamiento necesitamos partir de una serie cuyo índice sea numérico (la operación "sustracción" en la que se basa esta tercera opción no está soportada entre cadenas de texto).

In [ ]:
s = pd.Series([100, 200, 300, 400, 500], index = [10, 20, 30, 40, 50])
s

,0
10,100
20,200
30,300
40,400
50,500


Una serie cuyo índice está formado por múltiplos de 10. Generemos ahora una copia del mismo con el índice [20, 40, 19] aplicando como método de relleno "nearest".

In [ ]:
r = s.reindex([20, 40, 19], method = "nearest")
r

,0
20,200
40,400
19,200


El método ha incluido el índice 19 y le ha asignado el valor del índice más próximo (20), es decir, el valor de s[20] (200).

In [ ]:
r = s.reindex([20, 40, 11], method = "nearest")
r

,0
20,200
40,400
11,100


En este caso, el índice más próximo es 10, y el valor asignado es, por lo tanto, s[10]: 100.

## <font color='blue'>**Reindexación de dataframes**</font>

El método `pandas.DataFrame.reindex` ofrece una funcionalidad semejante a la disponible para series con la particularidad de que, en este caso, podemos reindexar por filas y/o por columnas. Por defecto, este método acepta una secuencia de etiquetas que determinarán qué filas se incluyen y en qué orden (es decir, por defecto la reindexación se aplica al eje 0).

In [ ]:
df = pd.DataFrame(np.arange(15).reshape([5, 3]),
                  index = ["a", "b", "c", "d", "e"],
                  columns = ["A", "B", "C"])
df

,A,B,C
a,0,1,2
b,3,4,5
c,6,7,8
d,9,10,11
e,12,13,14


In [ ]:
df.reindex(["d", "b"])

,A,B,C
d,9,10,11
b,3,4,5


In [ ]:
# df no ha cambiado
df

,A,B,C
a,0,1,2
b,3,4,5
c,6,7,8
d,9,10,11
e,12,13,14


En este ejemplo, partimos de un dataframe cuyo índice de filas tiene las etiquetas $a$, $b$, $c$, $d$ y $e$, y hemos indicado como nuevo índice las etiquetas $d$ y $b$ (en este orden), y son estas filas (en ese orden) las que se devuelven como resultado.

Este método permite especificar las etiquetas de filas como hemos visto, pasándoselas al método como primer argumento, o con el parámetro `index`.

In [ ]:
df.reindex(index = ["d", "b"])

,A,B,C
d,9,10,11
b,3,4,5


El parámetro `columns`, por su parte, permite especificar el nuevo índice de columnas:



In [ ]:
df.reindex(columns = ["A", "C"])

,A,C
a,0,2
b,3,5
c,6,8
d,9,11
e,12,14


Si utilizamos ambos parámetros al mismo tiempo, imponemos simultáneamente el nuevo índice para filas y columnas.

In [ ]:
df.reindex(index = ["a", "c", "f"], columns = ["A", "D", "C"])

,A,D,C
a,0.0,NaN,2.0
c,6.0,NaN,8.0
f,NaN,NaN,NaN


Podemos asignar a los valores inexistentes un valor concreto usando el parámetro `fill_value`, o podemos aplicar "lógica de relleno" con el parámetro `method`, permitiéndonos rellenar los valores inexistentes hacia adelante o hacia atrás.

Y, por supuesto, si los nuevos índices contienen los mismos elementos que los índices originales pero en otro orden, el resultado del método será equivalente al original ordenado según el nuevo criterio.

In [ ]:
df.reindex(index = ["a", "c", "b", "e", "d"], columns = ["B", "C", "A"])

,B,C,A
a,1,2,0
c,7,8,6
b,4,5,3
e,13,14,12
d,10,11,9


## <font color='blue'>**Método `set_index`**</font>

El método `pandas.DataFrame.set_index` fija una columna del dataframe como índice, descartando el índice existente.

In [ ]:
df = pd.DataFrame({
    "año": [2016, 2017, 2018],
    "mes": ["ene", "sep", "jun"],
    "ventas": [87, 34, 112]
})
df

,año,mes,ventas
0,2016,ene,87
1,2017,sep,34
2,2018,jun,112


Vemos que se ha asignado un índice automático. Si ejecutamos el método `set_index` indicando como argumento el campo "mes".

In [ ]:
df.set_index("mes")

,año,ventas
mes,,
ene,2016,87
sep,2017,34
jun,2018,112


Se fija dicha columna como índice y se elimina del conjunto de características. Aunque esta eliminación es el comportamiento por defecto, podemos controlarlo con el parámetro `drop`.

In [ ]:
df.set_index("mes", drop = False)

,año,mes,ventas
mes,,,
ene,2016,ene,87
sep,2017,sep,34
jun,2018,jun,112


### <font color='green'>Actividad 1</font>

Se tiene una serie temporal de ventas mensuales a lo largo de un año. Sin embargo, algunos meses faltan en el índice. Tu tarea es reindexar la serie para incluir todos los meses del año y tratar los valores faltantes.

```
months = ['Enero', 'Febrero', 'Marzo', 'Junio', 'Agosto', 'Noviembre', 'Diciembre']
sales = pd.Series([200, 220, 210, 230, 240, 250, 260], index=months)
```

1. Reindexa la serie sales para incluir todos los meses del año.
2. Rellena los valores faltantes con 0.
3. Calcula la media de ventas y reemplaza los valores faltantes con este valor en lugar de 0.






In [ ]:
months = ['Enero', 'Febrero', 'Marzo', 'Junio', 'Agosto', 'Noviembre', 'Diciembre']
sales = pd.Series([200, 220, 210, 230, 240, 250, 260], index=months)

In [ ]:
"""
1. Reindexa la serie sales para incluir todos los meses del año.
2. Rellena los valores faltantes con 0.
"""
resales = sales.reindex(index=['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo',
                               'Junio', 'Julio', 'Agosto', 'Septiembre',
                               'Octubre', 'Noviembre', 'Diciembre'],
                        fill_value=0)
display(resales)

,0
Enero,200
Febrero,220
Marzo,210
Abril,0
Mayo,0
Junio,230
Julio,0
Agosto,240
Septiembre,0
Octubre,0


In [ ]:
"""
3.
Calcula la media de ventas y reemplaza los valores faltantes con este valor
en lugar de 0.
"""

resales = sales.reindex(index=['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo',
                               'Junio', 'Julio', 'Agosto', 'Septiembre',
                               'Octubre', 'Noviembre', 'Diciembre'],
                        fill_value=sales.median())
display(resales)

,0
Enero,200.0
Febrero,220.0
Marzo,210.0
Abril,230.0
Mayo,230.0
Junio,230.0
Julio,230.0
Agosto,240.0
Septiembre,230.0
Octubre,230.0


<font color='green'>Fin actividad 1</font>

### <font color='green'>Actividad 2</font>

Supongamos que tienes dos DataFrames: uno con las ventas del primer semestre y otro con las ventas del segundo semestre. Quieres combinarlos en un solo DataFrame que contenga las ventas anuales, pero algunos productos están presentes en uno de los DataFrames y no en el otro.

```
products_first_semester = ['A', 'B', 'C', 'D']
sales_first_semester = pd.DataFrame({
    'sales': [100, 150, 130, 90]
}, index=products_first_semester)

products_second_semester = ['B', 'C', 'E', 'F']
sales_second_semester = pd.DataFrame({
    'sales': [110, 170, 120, 80]
}, index=products_second_semester)
```

1. Reindexa ambos DataFrames para que contengan todos los productos de ambos semestres.
2. Rellena los valores faltantes con la media de ventas del producto correspondiente (si un producto está presente en un semestre pero no en el otro, utiliza el valor del semestre presente).
3. Combina las ventas de ambos semestres en un nuevo DataFrame.






In [ ]:
products_first_semester = ['A', 'B', 'C', 'D']
sales_first_semester = pd.DataFrame({
    'sales': [100, 150, 130, 90]
}, index=products_first_semester)

products_second_semester = ['B', 'C', 'E', 'F']
sales_second_semester = pd.DataFrame({
    'sales': [110, 170, 120, 80]
}, index=products_second_semester)

In [ ]:
"""
1. Reindexa ambos DataFrames para que contengan todos los productos de ambos
semestres.
2. Rellena los valores faltantes con la media de ventas del producto
correspondiente (si un producto está presente en un semestre pero no en el
otro, utiliza el valor del semestre presente).
3. Combina las ventas de ambos semestres en un nuevo DataFrame.
"""
# Lista de todos los productos
products = sorted(set(products_first_semester + products_second_semester))

# Reindexación
sales_first_semester = sales_first_semester.reindex(products)

sales_second_semester = sales_second_semester.reindex(products)

# Concatenación
sales = pd.concat((sales_first_semester, sales_second_semester), axis=1,
                  join='outer')

# Rellenar datos
sales = sales.T.ffill().bfill().T

# Cambio nombres
sales.columns = ['sales 1st sem', 'sales 2nd sem']

# Agregar columna de la suma
sales['total_sales'] = sales.sum(axis=1)

# Despliegue
display(sales)

Ventas del primer semestre:


,sales
A,100.0
B,150.0
C,130.0
D,90.0
E,117.5
F,117.5


Ventas del segundo semestre:


,sales
A,120.0
B,110.0
C,170.0
D,120.0
E,120.0
F,80.0


<font color='green'>Fin actividad 2</font>